In [1]:
%%capture
!pip install sarus

In [2]:
from sarus import Client
client = Client(url='https://demo.sarus.tech:5000', email='datascientist@mycompany.com', password='datascientist1')
# Use your credentials here!

In [3]:
client.list_datasets()

[<Sarus Dataset slugname=imdb_extract id=61>,
 <Sarus Dataset slugname=patient_health_public_demo id=60>,
 <Sarus Dataset slugname=private_census id=59>]

In [4]:
remote_dataset = client.dataset(slugname="private_census") # Select an available dataset

In [5]:
remote_dataset.as_pandas().head()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income
0,31,Private,365752,Bachelors,13,Married-civ-spouse,Handlers-cleaners,Husband,White,Female,0,1707,55,United-States,>50K
1,46,Private,256057,Some-college,11,Married-civ-spouse,Exec-managerial,Not-in-family,White,Male,8614,0,50,United-States,<=50K
2,34,Private,128458,Masters,13,Never-married,Craft-repair,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,40,Federal-gov,357937,12th,9,Divorced,Sales,Husband,White,Male,0,0,45,United-States,>50K
4,28,Self-emp-not-inc,309187,Some-college,10,Married-civ-spouse,Craft-repair,Husband,White,Male,5178,1977,60,Holand-Netherlands,<=50K


In [6]:
import tensorflow as tf

def preprocess(batch):
    X = tf.stack([batch["age"], batch["education_num"], batch["hours_per_week"]], axis=1)
    X = tf.nn.relu(X)  # Replace encoded NaN by 0
    y = batch["income"]
    return X, y

In [7]:
tf_ds = remote_dataset.as_tensorflow().batch(16).map(preprocess)

In [8]:
from sarus.tensorflow import Model # Only changing the import line!

from tensorflow.keras.layers import Dense
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam

class DNN(Model): 
    def __init__(self):
        super().__init__()
        self.dense = Dense(units=10)
        self.dense2 = Dense(units=2)

    def call(self, x, training=False):
        return self.dense2(tf.nn.relu(self.dense(x)))

model = DNN()

model.compile(
    optimizer=Adam(learning_rate=1e-3),
    loss=SparseCategoricalCrossentropy(from_logits=True),
    metrics=["sparse_categorical_accuracy"],
)

In [9]:
model.fit(tf_ds, epochs=2, target_epsilon=1)

Epoch 1/2
312/312 [==============================] - 2s 1ms/step - loss: 4.1098 - sparse_categorical_accuracy: 0.5200
Epoch 2/2
312/312 [==============================] - 0s 1ms/step - loss: 2.5773 - sparse_categorical_accuracy: 0.5222

